In [54]:
import numpy as np
import pandas as pd
import xlwings as xw
import datetime

<b> Cập nhật link file path ở block dưới. </b>

In [55]:
updated_day = 'dd/mmm/yy'
gio_tre_sang = num_of_secs1
gio_tre_chieu = num_of_secs2

segments = [-1, 0, 1/24, 1.5/24, 2/24, 2.5/24, 3/24, 3.5/24, 4/24, 1]
segments_name = ['0.0hr', '<1.0hr', '1.0hr-1.5hr', '1.5hr-2.0hr', '2.0hr-2.5hr', '2.5hr-3.0hr', '3.0hr-3.5hr', '3.5hr-4.0hr', '>=4.0hr']

StaffList_file_path = input_file_path1
Headcount_file_path = input_file_path2

Talktime1_file_path = input_file_path3
Login1_file_path = input_file_path4
Calling1_file_path = input_file_path5

output_file_path = output_file_path1

### Read input files about Calling

In [56]:
Talktime1 = pd.read_csv(Talktime1_file_path)
Calling1 = pd.read_csv(Calling1_file_path)
Login1 = pd.read_csv(Login1_file_path)

### Read input files about Staffs

In [57]:
DSNV = pd.read_excel(StaffList_file_path, sheet_name='StaffList', index_col=0, usecols="A:J,L,S:U,W,AD")
HC = pd.read_excel(Headcount_file_path, index_col=False, usecols='C,D,E,F,G,H,I,J,K')

### Preprocess input files about Staffs

In [1]:
# DSNV
DSNV.loc[:,'Email'] = DSNV['Email'].str.strip()
DSNV_newest = DSNV.loc[DSNV.UPDATED_DAY == updated_day]
DSNV_newest.loc[:,'NAME'] = DSNV_newest['NAME'].str.strip()
DSNV_newest.loc[:,'Email'] = DSNV_newest['Email'].str.strip()
DSNV_newest.loc[:,'STATUS'] = DSNV_newest['STATUS'].str.strip()

# HC
HC['diem_danh'].fillna(0, inplace=True)

### Preprocess input_file_1 about Calling

In [67]:
Tt_raw = Talktime1

Tt_filter_outbound = Tt_raw['Interaction Type'] == 'Outbound'

Tt1 = Tt_raw[Tt_filter_outbound]

Tt1[['Họ và tên lót', 'Tên và Emails']] = Tt1.loc[:,'Agent Name'].str.split(pat=",", expand=True)
Tt1[['Tên', 'Emails']] = Tt1.loc[:,'Tên và Emails'].str.split(pat="(", expand=True)
Tt1.loc[:,'Họ và Tên'] = Tt1.loc[:,'Họ và tên lót'] + Tt1.loc[:,'Tên']
Tt1.loc[:,'Họ và Tên'] = Tt1.loc[:,'Họ và Tên'].str.strip()
Tt1.loc[:,'Emails'] = Tt1.loc[:,'Emails'].str.rstrip(')')
Tt1.drop(columns=['Họ và tên lót', 'Tên và Emails', 'Tên', 'Agent Name', 'Interaction Type', 'Media Type'], axis=1, inplace=True)
Tt1.drop_duplicates(inplace=True)

### Preprocess input_file_2 about Calling

In [68]:
Log1 = Login1

Log1[['Họ và tên lót', 'Tên và Emails']] = Log1.loc[:,'Agent Name'].str.split(pat=",", expand=True)
Log1[['Tên', 'Emails']] = Log1.loc[:,'Tên và Emails'].str.split(pat="(", expand=True)
Log1['Họ và Tên'] = Log1.loc[:,'Họ và tên lót'] + Log1.loc[:,'Tên']
Log1.loc[:,'Họ và Tên'] = Log1.loc[:,'Họ và Tên'].str.strip()
Log1.loc[:,'Emails'] = Log1.loc[:,'Emails'].str.rstrip(')')
Log1.drop(columns=['Họ và tên lót', 'Tên và Emails', 'Tên', 'Agent Name', 'Agent Group', 'Media Type'], axis=1, inplace=True)
Log1.drop_duplicates(subset=['Emails', 'Start Timestamp', 'End Timestamp'], keep='last', inplace=True)

check_Log1 = Log1.merge(DSNV_newest, how = "left", left_on='Emails', right_on='Email')
check_Log1.columns

check_Log1 = check_Log1.drop(['CODE', 'MAP FUNNEL', 'USING CODE', 'POSITION', 'UPDATED_DAY'], axis=1)
# check_Tt1.dropna(subset=['USING EXT'], inplace=True)
check_Log1.fillna('na', inplace=True)
check_Log1['Start Timestamp'] = pd.to_datetime(check_Log1['Start Timestamp'])
check_Log1['End Timestamp'] = pd.to_datetime(check_Log1['End Timestamp'])


login = Log1.groupby(['Day', 'Emails','Họ và Tên'])['Start Timestamp']
login_time = login.min()
login_time = pd.DataFrame(login_time)
login_time.reset_index(inplace=True)

logoff = Log1.groupby(['Day', 'Emails','Họ và Tên'])['End Timestamp']
logoff_time = logoff.max()
logoff_time = pd.DataFrame(logoff_time)
logoff_time.reset_index(inplace=True)

login_logoff_time = pd.merge(login_time, logoff_time, how='outer')
# login_logoff_time = pd.concat([login_time, logoff_time], axis=1)
# _, i = np.unique(login_logoff_time.columns, return_index=True)
# login_logoff_time = login_logoff_time.iloc[:, i]

login_logoff_time['End Timestamp'] = pd.to_datetime(login_logoff_time['End Timestamp'])
login_logoff_time['Start Timestamp'] = pd.to_datetime(login_logoff_time['Start Timestamp'])
login_logoff_time['Worktime'] = login_logoff_time['End Timestamp'] - login_logoff_time['Start Timestamp'] - pd.to_timedelta(1, unit='h')

### Preprocess input_file_3 about Calling

In [69]:
Calling = Calling1

Calling_filter_outbound = Calling['Interaction Type'] == 'Outbound'
Calling = Calling[Calling_filter_outbound]
Calling.drop(['GUID', 'Interaction Type', 'From', 'TeamLeadName', 'Business Result', 'Customer ID'], axis=1, inplace=True)
Calling['Start Timestamp'] = pd.to_datetime(Calling['Start Timestamp'])
Calling['Calling Date'] = Calling['Start Timestamp'].dt.date

Calling['after_12h'] = (Calling['Start Timestamp'] - pd.to_datetime(Calling['Start Timestamp'].dt.date)) > pd.to_timedelta(43200, unit='s')

Calling.loc[(Calling['after_12h'] == False), '12h'] = 'Morning'
Calling.loc[(Calling['after_12h'] == True), '12h'] = 'Afternoon'

first_calls = Calling.groupby(['Calling Date', 'Handling Resource','12h'])['Start Timestamp']
firstcalls = first_calls.min()
firstcalls = pd.DataFrame(firstcalls)
firstcalls.reset_index(inplace=True)

A1 = firstcalls['12h'] == 'Morning'
A2 = firstcalls['12h'] == 'Afternoon'

B1 = (firstcalls['Start Timestamp'] - pd.to_datetime(firstcalls['Start Timestamp'].dt.date)) > pd.to_timedelta(gio_tre_sang, unit='s')
B2 = (firstcalls['Start Timestamp'] - pd.to_datetime(firstcalls['Start Timestamp'].dt.date)) > pd.to_timedelta(gio_tre_chieu, unit='s')

firstcalls.loc[A1 & B1 | A2 & B2, 'Tre'] = 'Trễ'

### Process output_sheets for output_file

In [72]:
# sheet Calling
check_Calling1 = Calling.merge(DSNV_newest, how = "left", left_on='Handling Resource', right_on='Email')

check_Calling1 = check_Calling1.drop(['CODE', 'MAP FUNNEL', 'USING CODE', 'POSITION', 'STATUS', 'UPDATED_DAY'], axis=1)
                               
# sheet check_Tt1
check_Tt1 = Tt1.merge(DSNV_newest, how = "left", left_on='Emails', right_on='Email')

check_Tt1 = check_Tt1.drop(['CODE', 'MAP FUNNEL', 'USING CODE', 'POSITION', 'STATUS', 'UPDATED_DAY'], axis=1)
# check_Tt1.dropna(subset=['USING EXT'], inplace=True)
check_Tt1.fillna('na', inplace=True)

check_Tt1.rename(columns={'Accepted':'Answered Calls', 'Engage Time':'Talk Time'}, inplace=True)
check_Tt1['Talk Time'] = check_Tt1['Talk Time']/(3600*24)
check_Tt1['Day'] = pd.to_datetime(check_Tt1['Day'])


check_Tt1_HC = pd.merge(check_Tt1, HC, how = 'left', left_on=['Day', 'WORKING_CODE'], right_on=['DATE', 'TSA CODE'])
check_Tt1_HC.drop(['CAMPAIGNS', 'TEAMLEAD', 'SS', 'TSA CODE', 'DATE', 'STATUS', 'NOTE'], axis=1, inplace=True)
check_Tt1 = check_Tt1[['Day', 'MNV', 'WORKING_CODE', 'Họ và Tên', 'Answered Calls', 'Talk Time', 'TYPE', 'USING EXT', 'TL', 'SUP', 'CAMPAIGN', 'Emails']]

# check_Tt1_campaign_HCchinh
check_Tt1_campaign_HCchinh = check_Tt1_HC[check_Tt1_HC.TYPE == 'HC chính'].pivot_table(index=['Day', 'CAMPAIGN', 'SUP'], values=['Talk Time', 'diem_danh'], aggfunc={'Talk Time':np.mean, 'diem_danh':'sum'})

# check_Tt1_campaign_Backup
check_Tt1_campaign_Backup = check_Tt1_HC[check_Tt1_HC.TYPE == 'Backup'].pivot_table(index=['Day', 'CAMPAIGN', 'SUP'], values=['Talk Time', 'diem_danh'], aggfunc={'Talk Time':np.mean, 'diem_danh':'sum'})

# check_Tt1_segments_SUP & check_Tt1_segments_TL
check_Tt1_HC['Phân khúc'] = pd.cut(check_Tt1['Talk Time'], segments , labels = segments_name)
check_Tt1_HC['Phân khúc'].fillna('0.0hr', inplace=True)

check_Tt1_segments_SUP = check_Tt1_HC[check_Tt1_HC.TYPE == 'HC chính'].pivot_table(index=['Day', 'SUP'], columns='Phân khúc', values='Talk Time', aggfunc=['count'])
check_Tt1_segments_TL = check_Tt1_HC[check_Tt1_HC.TYPE == 'HC chính'].pivot_table(index=['Day', 'TL', 'SUP'], columns='Phân khúc', values='Talk Time', aggfunc=['count'])

# sheet check_worktime
check_worktime = login_logoff_time.merge(DSNV_newest, how = "left", left_on='Emails', right_on='Email')
check_worktime = check_worktime.drop(['CODE', 'MAP FUNNEL', 'USING CODE', 'POSITION', 'UPDATED_DAY', 'Email'], axis=1)
check_worktime.fillna('na', inplace=True)

# sheet check_firstcalls 
check_firstcalls = firstcalls.merge(DSNV_newest, how = "left", left_on='Handling Resource', right_on='Email')
check_firstcalls = check_firstcalls.drop(['CODE', 'MAP FUNNEL', 'USING CODE', 'POSITION', 'UPDATED_DAY', 'Email'], axis=1)
check_firstcalls.fillna('na', inplace=True)

# sheet check_tre
check_tre = check_firstcalls[check_firstcalls.Tre == 'Trễ'].pivot_table(index=['Calling Date', 'CAMPAIGN', 'SUP'], values=['Handling Resource'], aggfunc='count')

### Export output_file (Excel) with multiple sheets processed above

In [77]:
with pd.ExcelWriter(output_file_path) as writer:
    check_Tt1_segments_SUP.to_excel(writer, sheet_name='Phan khuc Tt theo SUP') # DONE
    check_Tt1_segments_TL.to_excel(writer, sheet_name='Phan khuc Tt theo TL') # DONE
    check_Tt1_campaign_HCchinh.to_excel(writer, sheet_name='Talktime_HCchinh') # DONE
    check_Tt1_campaign_Backup.to_excel(writer, sheet_name='Talktime_Backup') # DONE
    check_tre.to_excel(writer, sheet_name='Calls_Tre') # DONE
    check_worktime.to_excel(writer, sheet_name='Login-Logoff') # DONE
    check_firstcalls.to_excel(writer, sheet_name='First_Calls') # DONE
    check_Tt1.to_excel(writer, sheet_name='Talktime_Database') # DONE    